# Evolución diferencial

La evolución diferencial (ED) es un algoritmo diseñado para optimizar funciones cuyo dominio es un subconjunto de $\mathbb{R}^n$. Cada individuo en la población es una vector de dimensipn $n$ que representa una posible solución al problema. La evolución diferencial está basada en la idea de tomar la diferencia entre dos individuos y agregar una versión escalada de esta diferencia a un tercer individuo con el fin de obtener una nueva posible solución al problema de optimización, como se muestra en la siguiente ilustración:

<img src="../../files/misc/idea_ed.png" width=400/>

Dos individuos distintos $\mathbf{x_{r2}}, \mathbf{x_{r3}}$ se seleccionan al azar y su diferencia $\mathbf{x_{r2} - x_{r3}}$ es calculada. Esta diferencia es escalada por un factor (escalar) $F \in \mathbb{R}$ y se agrega a otro individuo $\mathbf{x_{r1}}$ (distinto a $\mathbf{x_{r2}}$ y a $\mathbf{x_{r3}}$), lo que da como resultado el nuevo individuo

$$
\mathbf{v_i} = \mathbf{x_{r1}} + F(\mathbf{x_{r2} - x_{r3}})
$$

Una vez obteniendo el vector $\mathbf{v_i}$, este es combinado con otro individuo $\mathbf{x_i}$ con $i \neq r1, r2, r3$, para crear el vector $\mathbf{u_i}$. La combinación se realiza entrada por entrada, es decir, si $\mathbf{v_i} = (v_{i1}, v_{i2}, \ldots, v_{in})$ y $\mathbf{x_i} = (x_{i1}, x_{i2}, \ldots, x_{in})$, entonces $\mathbf{u_i} = (u_{i1}, u_{i2}, \ldots, u_{in})$ en donde 

$$
u_{ij} = 
\begin{cases}
v_{ij} \mbox{ Si } r_{cj} < c \mbox{ o } j = \mathcal{J} \\
x_{ij} \mbox{ en otro caso }
\end{cases}
$$

en donde $r_{cj}$ es un número aleatorio obtenido de la distribución uniforme $(0,1)$ y $c$ es la tasa de cruza, $c\in(0,1)$.

El proceso anterior se repite hasta obtener $N$ (tamaño de la población) individuos $\mathbf{u_i}$ los cuales se compararán con los individuos $\mathbf{x_i}$ (población anterior). $\mathcal{J}$ es un número aleatorio elegido dentro del conjunto $\{1,2, \ldots, n\}$.

El algoritmo completo de la ED se ilustra a continuación (se intenta resolver el problema de minimizar la función $f$)

<img src="../../files/misc/algoritmo_ed.png" width=700/>


## Función objetivo

En este proyecto se busca minimizar la siguiente función (la función de Ackley)

$$
f(\mathbf{x}) = 20 + e - 20 exp\left(-0.2 \sum_{i=1}^{n} \dfrac{x_{i}^2}{n}\right) - exp\left( \sum_{i=1}^{n} \dfrac{cos(2 \pi x_i)}{n} \right)
$$

restringiendo cada componente del vector $\mathbf{x}$, $x_i \in [-30, 30]$.

El óptimo es $\mathbf{x^*} = (0,0, \ldots, 0)$. 

## Problema

Programe el algoritmo de evolución diferencial con el fin de minimizar la función de Ackley. Como condición de paro utilice 6000 generaciones. Imprima el valor del mejor individuo en cada generación, el mejor individuo es aquel con menor valor $f(\mathbf{x})$.

## Hiperparámetros

Utilice los siguientes valores para los hiperparámetros

* $F = 0.2$
* $c = 0.3$
* Tamaño de la población $N = 100$.
* Dimensión del problema $n = 10$.

## Sugerencias

* Lea la documentación de ```numpy.random.choice```
* Lea la documentación de ```numpy.random.uniform```
* Lea la documentación de argmin ```numpy.argmin```

In [1]:
import numpy as np

In [2]:
def inicializa_poblacion(num_ind = 100, dim = 10, lim_inf = -30.0, lim_sup = 30.0):
    '''
    Función para inicializar la población
    
    ENTRADA
    num_ind: Entero positivo (número de individuos en la población)
    
    dim: Entero positivo (dimensión de cada individuo)
    
    lim_inf, lim_sup: Floats
    
    SALIDA
    numpy array con shape (num_ind, dim) y entradas en [lim_inf, lim_sup]
    '''
    
    return np.random.uniform(low = lim_inf, high = lim_sup, size = (num_ind, dim))

In [3]:
def ackley(x):
    '''
    Función de Ackley
    
    ENTRADA
    x: numpy array
    
    SALIDA
    float
    '''
    return 20 + np.exp(1) - 20 * np.exp(-0.2 * (x**2).mean()) - np.exp( np.cos(2*np.pi*x).mean() )

In [4]:
def evolucion_diferencial(f_obj, num_ind = 100, dim = 10, lim_inf = -30, 
                          lim_sup = 30, F = 0.65, c = 0.55, n_iter = 300):
    '''
    Implementación del algoritmo de evolución diferencial
    
    ENTRADA
    f_obj: Función objetivo
    
    num_ind: Número de individuos en la población
    
    dim: Dimensión del problema
    
    lim_inf, lim_sup: Floats (cota inferior y superior de cada dimensión)
    
    F: float
    
    c: Float en (0,1)
    
    n_iter: Entero positivo (número de iteraciones)
    
    SALIDA
    numpy array que representa el mejor individuo después de n_iter iteraciones
    '''
    np.random.seed(54321)
    #Inicializa población
    poblacion = inicializa_poblacion(num_ind, dim, lim_inf, lim_sup)
    
    #Obtiene el valor de f_obj en cada individuo
    f_pob = np.apply_along_axis(func1d = f_obj, axis = 1, arr = poblacion)
    
    #Criterio de terminación
    for iteracion in range(n_iter):
        
        nueva_pob = []
        
        #Para cada individuo i
        for i in range(num_ind):
            
            x_i = poblacion[i]
            posibles_x = list(range(num_ind))
            posibles_x.pop(i) #Para que r1,r2 y r3 no sean igual a i
            
            #Elige los índices r1, r2 y r3 (sin reemplazo)
            r = np.random.choice(posibles_x, size = 3, replace = False)
            
            #Mutant vector
            v_i = poblacion[r[0]] + F * (poblacion[r[1]] - poblacion[r[2]])
            
            #Muta v_i entrada por entrada
            #J_r = np.random.choice(range(dim), size = 1)[0]
            
            for d in range(dim):
                
                J_r = np.random.choice(range(dim), size = 1)[0]
                #Aleatorio uniforme en (0 , 1)
                unif = np.random.uniform(0, 1, 1)[0]
                
                if unif >= c and d != J_r:
                    v_i[d] = x_i[d]
                    
            #Aplica restricciones de que cada entrada debe estar en [lim_inf, lim_sup]
            for d in range(dim):
                if v_i[d] < lim_inf:
                    v_i[d] = np.random.uniform(-30,30, 1)[0]
                if v_i[d] > lim_sup:
                    v_i[d] = np.random.uniform(-30,30, 1)[0]
            #Agrega el individuo v_i a la nueva población
            nueva_pob.append(v_i)
            
        #Evalua f_obj en cada individuo de la nueva población
        nueva_pob = np.array(nueva_pob)
        f_nueva_pob = np.apply_along_axis(func1d = f_obj, axis = 1, arr = nueva_pob)
        
        #Selecciona los mejores individuos entre las dos poblaciones
        for k in range(num_ind):
            if f_nueva_pob[k] < f_pob[k]:
                poblacion[k] = nueva_pob[k]
                
        #Obtiene el valor de f_obj en cada individuo 
        #después obtiene el mejor individuo
        f_pob = np.apply_along_axis(func1d = f_obj, axis = 1, arr = poblacion)
        indice_mejor = np.argmin(f_pob)
        mejor_ind = poblacion[indice_mejor]
        
        if iteracion % 100 == 0:
            print('Para la iteración {} el mejor individuo es {} '.format(iteracion,mejor_ind))
            print('Su valor en f_obj es {} '.format(f_pob[indice_mejor]))
            print('---'*30)
        
    return mejor_ind

In [5]:
evolucion_diferencial(ackley, n_iter = 6000, c = 0.3, F = 0.2)

Para la iteración 0 el mejor individuo es [-25.7550981   27.07807893   6.11963343  -0.95520854  25.94850017
   5.75359023  17.41904145  16.11313277  15.98531842  16.91615244] 
Su valor en f_obj es 21.0158585532104 
------------------------------------------------------------------------------------------
Para la iteración 100 el mejor individuo es [  5.01410429 -13.04368132  15.96721446 -13.96792939 -15.94711901
  26.04270035  10.04595645  24.01902955  -7.9953666    4.01909856] 
Su valor en f_obj es 20.061634628766203 
------------------------------------------------------------------------------------------
Para la iteración 200 el mejor individuo es [  5.01410429 -13.04368132  15.96721446 -13.96792939 -15.94711901
  26.04270035  10.04595645  24.01902955  -7.9953666    4.01909856] 
Su valor en f_obj es 20.061634628766203 
------------------------------------------------------------------------------------------
Para la iteración 300 el mejor individuo es [  7.03103466  -3.98330956  17

Para la iteración 2800 el mejor individuo es [ 0.80234121 -0.99318105 -0.0788099   0.2128129  -0.6020133  -0.78334034
 -0.3428172   0.39537518  1.82336395  0.80034627] 
Su valor en f_obj es 4.161630997359446 
------------------------------------------------------------------------------------------
Para la iteración 2900 el mejor individuo es [ 0.80234121 -0.99318105 -0.0788099   0.2128129  -0.6020133  -0.78334034
 -0.3428172   0.39537518  1.82336395  0.80034627] 
Su valor en f_obj es 4.161630997359446 
------------------------------------------------------------------------------------------
Para la iteración 3000 el mejor individuo es [ 0.80234121 -0.99318105 -0.0788099   0.2128129  -0.6020133  -0.78334034
 -0.3428172   0.39537518  1.82336395  0.19750831] 
Su valor en f_obj es 3.9492996228884594 
------------------------------------------------------------------------------------------
Para la iteración 3100 el mejor individuo es [ 0.80234121 -0.99318105 -0.0788099   0.2128129  -0.60

Para la iteración 5400 el mejor individuo es [ 1.39968073e-09 -5.85249596e-10 -2.62592286e-09  3.65693919e-09
  2.60035414e-09 -3.19573607e-09 -8.92116514e-10 -1.30534973e-09
  8.70513338e-10  2.59919322e-11] 
Su valor en f_obj es -4.440892098500626e-16 
------------------------------------------------------------------------------------------
Para la iteración 5500 el mejor individuo es [ 1.39968073e-09 -5.85249596e-10 -2.62592286e-09  3.65693919e-09
  2.60035414e-09 -3.19573607e-09 -8.92116514e-10 -1.30534973e-09
  8.70513338e-10  2.59919322e-11] 
Su valor en f_obj es -4.440892098500626e-16 
------------------------------------------------------------------------------------------
Para la iteración 5600 el mejor individuo es [ 1.39968073e-09 -5.85249596e-10 -2.62592286e-09  3.65693919e-09
  2.60035414e-09 -3.19573607e-09 -8.92116514e-10 -1.30534973e-09
  8.70513338e-10  2.59919322e-11] 
Su valor en f_obj es -4.440892098500626e-16 
-----------------------------------------------------

array([ 1.39968073e-09, -5.85249596e-10, -2.62592286e-09,  3.65693919e-09,
        2.60035414e-09, -3.19573607e-09, -8.92116514e-10, -1.30534973e-09,
        8.70513338e-10,  2.59919322e-11])